## Imports and data prep

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv('data-week-3.csv')
 
df.columns = df.columns.str.lower().str.replace(' ', '_')
 
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
 
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')
 
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)
 
df.churn = (df.churn == 'yes').astype(int)
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,1


## Split the data and train the model

In [3]:
# Next, we use the ‘train_test_split’ function to split the datasets into ‘full_train’ (80%) and ‘test’ in the first step. 
# In the second step, ‘full_train’ is further divided into two datasets for training and validation. 
# Ultimately, we achieve a split of the initial data into a 60%-20%-20% ratio, where 60% is used for training, 
# and 20% each is allocated for validation and testing. 
# The parameter ‘random_state=1’ ensures that the random split is reproducible.

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
 
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
 
y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values
 
del df_train['churn']
del df_val['churn']
del df_test['churn'] 

df_train.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,yes,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15
1,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,yes,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55
2,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,no,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65
3,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,no,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10
4,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50


In [5]:
# Define two features lists, ‘numerical’ and ‘categorical’, which contain the relevant column names. 
# The ‘numerical’ array contains the names of all numerical columns, while the ‘categorical’ 
# array contains the names of all categorical columns.

numerical = ['tenure', 'monthlycharges', 'totalcharges']
 
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
       'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']

In [6]:
# Next, we create a DictVectorizer instance. We then transform the dataframe into dictionaries, 
# and using the ‘fit_transform(train_dict)’ function, we train the DictVectorizer. This step involves 
# showing the DictVectorizer how the data is structured, allowing it to distinguish column names and 
# values and perform one-hot encoding based on this information. Importantly, the DictVectorizer is 
# smart enough to distinguish between categorical values and numeric values, so numeric values are 
# ignored during one-hot encoding. The ‘transform’ part of this process converts the dictionary 
# into a vector or matrix suitable for machine learning.

# After preparing the data, we move on to model creation. In this case, a Logistic Regression model is used. 
# The ‘model.fit’ function is then employed to train the model on the training data.

dv = DictVectorizer(sparse=False)
 
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
 
model = LogisticRegression()
model.fit(X_train, y_train)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [7]:
# We can then proceed to validate the trained model using the validation data. 
# To do this, we need to prepare the validation DataFrame in the same way as shown 
# for the training DataFrame. This involves transforming it into dictionaries and 
# applying the ‘transform’ function of the DictVectorizer. However, during validation, 
# we only need to use the ‘transform’ function of the DictVectorizer since it already 
# knows the data structure. In the case of validation, we are primarily interested 
# in the transformed output, which serves as input for prediction.

# For prediction, we use the ‘predict_proba’ function of the model, which provides 
# us with probabilities in two columns. Here, we are interested in the second 
# column. We evaluate the model’s performance using a threshold of ‘>=0.5’. 
# The ‘churn_decision’ variable contains ‘True’ for any value in the prediction 
# greater than or equal to the threshold, and ‘False’ otherwise. We calculate the 
# accuracy using the ‘mean’ function, and in this case, it is approximately 80%.

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
 
y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
(y_val == churn_decision).mean()
 
# Output: 0.8034066713981547

np.float64(0.8034066713981547)

In [8]:
# calc another way to double-check myself:
churn_decision2 = model.predict(X_val)
(y_val == churn_decision2).mean()
# same output 0.8034066713981547

np.float64(0.8034066713981547)